In [26]:
##############################################################################################
## set up the reference model for demo purposes
##############################################################################################

import numpy as np
from sklearn import mixture

np.random.seed(1)

g = mixture.GMM(covariance_type="full", n_components=2)

dimensions = 2

obs = np.concatenate((np.random.randn(100, dimensions), 10 + np.random.randn(300, dimensions)))

g.fit(obs) 

print "mixture component weights\n%s\n" % g.weights_

print "mixture component mean vectors\n%s\n" % g.means_

print "mixture component covariance matrices\n%s" % g.covars_

mixture component weights
[ 0.75  0.25]

mixture component mean vectors
[[  9.95979962  10.04835949]
 [  0.22665544  -0.01327782]]

mixture component covariance matrices
[[[ 1.04862299 -0.10230272]
  [-0.10230272  1.05411687]]

 [[ 0.73906418 -0.11445233]
  [-0.11445233  0.89051922]]]


In [27]:
##############################################################################################
## method #1: directly compute Mahalanobis distance from stored precision matrix and scale
## by stored determinant
##############################################################################################

from scipy import linalg
from scipy import constants

numberOfMixtureComponents = g.n_components

np.random.seed(1)

sample = (3 + np.random.randn(1, dimensions))[0]

print "sample\n%s" % sample

print

print "scipy GMM pdf\n%s" % g.predict_proba([ sample ])

print "scipy GMM log pr\n%s" % g.score([ sample ])

# store the pdf of each mixture component
gaussianPDF = []

# store the sum of the pdfs
gaussianPDFSum = 0

for mixtureComponentIndex in range(numberOfMixtureComponents):
    print "\nmixture component %i" % mixtureComponentIndex
    
    # 1. get the given parameters for the current mixture component
    mixtureWeight = g.weights_[mixtureComponentIndex]

    print "  mixture weight %f" % mixtureWeight
    
    meanVector = g.means_[mixtureComponentIndex]
    
    print "  mean vector\n%s" % meanVector
    
    covarianceMatrix = g.covars_[mixtureComponentIndex]
    
    precisionMatrix = linalg.inv(covarianceMatrix)
    
    print "  precision matrix\n%s" % precisionMatrix
    
    determinant = linalg.det(covarianceMatrix)
    
    numberOfDimensions = len(meanVector)
    
    # 2. center the data sample:
    centeredSample = []
    
    for i in range(numberOfDimensions):
        sampleValue = sample[i]
        
        mean = meanVector[i]
        
        centeredSampleValue = sampleValue - mean
        
        centeredSample.append(centeredSampleValue)
    
    # 3. form the scale coefficient
    scale = np.power((2 * constants.pi), numberOfDimensions)
    
    scale = scale * determinant
    
    scale = 1.0 / np.sqrt(scale)
    
    print "  scale coefficient %f" % scale
        
    # 4. perform the vector-matrix-vector Mahalanobis distance calculation
    distanceTemp = []
    
    for i in range(numberOfDimensions):
        distanceValue = 0
        
        for j in range(numberOfDimensions):
            centeredSampleValue = centeredSample[j]
            
            precisionValue = precisionMatrix[j][i]
            
            tempValue = centeredSampleValue * precisionValue
            
            distanceValue = distanceValue + tempValue
                        
        distanceTemp.append(distanceValue)
        
    distance = 0
    
    for i in range(numberOfDimensions):
        centeredSampleValue = centeredSample[i]
        
        distanceValue = distanceTemp[i]
        
        tempValue = centeredSampleValue * distanceValue
        
        distance = distance + tempValue
        
    distance = - 0.5 * distance

    # 5. obtain the pdf
    pdf = scale * np.exp(distance)
    
    # 6. obtain the posterior pdf
    posteriorPDF = mixtureWeight * pdf
    
    # 7. increment the pdf sum
    gaussianPDFSum = gaussianPDFSum + posteriorPDF
    
    # 8. keep track of each pdf
    gaussianPDF.append(posteriorPDF)

print
    
# normalize each pdf by the sum for presentation to compare against scipy results
print "calculated Gaussian pdf\n%s" % (gaussianPDF / gaussianPDFSum)

# don't normalize each pdf by the sum
print "calculated pr\n%s" % gaussianPDFSum

print "calculated log pr\n%s" % np.log(gaussianPDFSum)

sample
[ 4.62434536  2.38824359]

scipy GMM pdf
[[  3.94305515e-12   1.00000000e+00]]
scipy GMM log pr
[-21.53236115]

mixture component 0
  mixture weight 0.750000
  mean vector
[  9.95979962  10.04835949]
  precision matrix
[[ 0.96274704  0.09343522]
 [ 0.09343522  0.95772936]]
  scale coefficient 0.152101

mixture component 1
  mixture weight 0.250000
  mean vector
[ 0.22665544 -0.01327782]
  precision matrix
[[ 1.38053969  0.1774313 ]
 [ 0.1774313   1.14574442]]
  scale coefficient 0.198163

calculated Gaussian pdf
[  3.94305515e-12   1.00000000e+00]
calculated pr
4.45260707191e-10
calculated log pr
-21.5323611465


In [23]:
##############################################################################################
## calculate the constant values (these will all be hard-coded at compile-time) for method #2
##############################################################################################

from scipy import linalg
from scipy import constants

numberOfMixtureComponents = g.n_components

gmmParams = []

for mixtureComponentIndex in range(numberOfMixtureComponents):
    print "\nmixture component index %i" % mixtureComponentIndex
    
    # 1. setup the parameters for the current mixture component
    mixtureWeight = g.weights_[mixtureComponentIndex]

    print "  mixture weight scalar %f" % mixtureWeight
    
    meanVector = g.means_[mixtureComponentIndex]
    
    print "  mean vector\n%s" % meanVector
    
    covarianceMatrix = g.covars_[mixtureComponentIndex]
    
    numberOfDimensions = len(meanVector)
    
    covarianceUpperCholesky = linalg.cholesky(covarianceMatrix)
    
    inverseCovarianceUpperCholesky = linalg.inv(covarianceUpperCholesky)
    
    print "  covariance matrix upper cholesky decomposition inverse matrix\n%s" % inverseCovarianceUpperCholesky
    
    diagProd = 1.
    
    for dimension in range(numberOfDimensions):
        diag = covarianceUpperCholesky[dimension][dimension]
        
        diagProd = diagProd * diag
    
    scaleCoefficient = np.power((2 * constants.pi), numberOfDimensions)
    
    scaleCoefficient = 1.0 / np.sqrt(scaleCoefficient)
    
    scaleCoefficient = scaleCoefficient / diagProd
    
    print "  scale coefficient scalar %f" % scaleCoefficient
        
    # 2. store the parameters for the current mixture component
    params = {
        "mixtureWeight": mixtureWeight,
        "meanVector": meanVector,
        "inverseCovarianceUpperCholesky": inverseCovarianceUpperCholesky,
        "scaleCoefficient": scaleCoefficient
    }
    
    gmmParams.append(params)


mixture component index 0
  mixture weight scalar 0.750000
  mean vector
[  9.95979962  10.04835949]
  covariance matrix upper cholesky decomposition inverse matrix
[[ 0.97654062  0.0954749 ]
 [ 0.          0.97863648]]
  scale coefficient scalar 0.152101

mixture component index 1
  mixture weight scalar 0.250000
  mean vector
[ 0.22665544 -0.01327782]
  covariance matrix upper cholesky decomposition inverse matrix
[[ 1.16321213  0.16576253]
 [ 0.          1.07039451]]
  scale coefficient scalar 0.198163


In [29]:
##############################################################################################
## method #2: use the stored inverse of the upper Cholesky decomposition of the covariance
## matrix to compute the distance to each mixture component (linear algebra tricks)
##############################################################################################

np.random.seed(1)

sample = (3 + np.random.randn(1, dimensions))[0]

print "sample\n%s" % sample

print

print "scipy GMM pdf\n%s" % g.predict_proba([ sample ])

print "scipy GMM log pr\n%s" % g.score([ sample ])

print "scipy GMM pr\n%s" % np.exp(g.score([ sample ]))

# store the pdf of each mixture component
gaussianPDF = []

# store the sum of the pdfs
gaussianPDFSum = 0

for mixtureComponentIndex in range(numberOfMixtureComponents):
    # 1. get the given parameters for this mixture component:
    params = gmmParams[mixtureComponentIndex]
    
    mixtureWeight = params["mixtureWeight"]

    meanVector = params["meanVector"]
        
    numberOfDimensions = len(meanVector)
    
    inverseCovarianceUpperCholesky = params["inverseCovarianceUpperCholesky"]
    
    scaleCoefficient = params["scaleCoefficient"]
    
    # 2. center the data sample:
    centeredSample = []
    
    for i in range(numberOfDimensions):
        sampleValue = sample[i]
        
        meanValue = meanVector[i]
        
        centeredSampleValue = sampleValue - meanValue
        
        centeredSample.append(centeredSampleValue)
    
    # 3. perform the Mahalanobis distance calculation using the 
    # inverse of the Cholesky decomposition of the covariance matrix
    # see mvnpdf.m for method
    distance = 0
    
    for i in range(numberOfDimensions):
        distanceValue = 0
        
        for j in range(numberOfDimensions):
            centeredSampleValue = centeredSample[j]
            
            inverseCovarianceUpperCholeskyValue = inverseCovarianceUpperCholesky[j][i]
            
            tempValue = centeredSampleValue * inverseCovarianceUpperCholeskyValue
            
            distanceValue = distanceValue + tempValue
            
        distanceValue = distanceValue * distanceValue
                        
        distance = distance + distanceValue
                
    distance = - 0.5 * distance

    # 4. obtain the pdf
    pdf = scaleCoefficient * np.exp(distance)
    
    # 5. obtain the posterior pdf
    posteriorPDF = mixtureWeight * pdf
    
    # 6. increment the pdf sum
    gaussianPDFSum = gaussianPDFSum + posteriorPDF
    
    # 7. keep track of each pdf
    gaussianPDF.append(posteriorPDF)

print
    
# normalize each pdf by the sum for presentation to compare against scipy results
print "calculated Gaussian pdf\n%s" % (gaussianPDF / gaussianPDFSum)

# don't normalize each pdf by the sum
print "calculated pr\n%s" % gaussianPDFSum

print "calculated log pr\n%s" % np.log(gaussianPDFSum)

sample
[ 4.62434536  2.38824359]

scipy GMM pdf
[[  3.94305515e-12   1.00000000e+00]]
scipy GMM log pr
[-21.53236115]
scipy GMM pr
[  4.45260707e-10]

calculated Gaussian pdf
[  3.94305515e-12   1.00000000e+00]
calculated pr
4.45260707191e-10
calculated log pr
-21.5323611465
